In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tinyllama-final/adapter_model.safetensors
/kaggle/input/tinyllama-final/trainer_state.json
/kaggle/input/tinyllama-final/training_args.bin
/kaggle/input/tinyllama-final/adapter_config.json
/kaggle/input/tinyllama-final/README.md
/kaggle/input/tinyllama-final/tokenizer.json
/kaggle/input/tinyllama-final/tokenizer_config.json
/kaggle/input/tinyllama-final/scaler.pt
/kaggle/input/tinyllama-final/scheduler.pt
/kaggle/input/tinyllama-final/special_tokens_map.json
/kaggle/input/tinyllama-final/optimizer.pt
/kaggle/input/tinyllama-final/rng_state.pth
/kaggle/input/tinyllama-final/tokenizer.model


In [2]:
pip install transformers peft accelerate faiss-cpu sentence-transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 62.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 3.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.1 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled n

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

base_model = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
adapter_path = "/kaggle/input/tinyllama-final"  # your LoRA fine-tuned checkpoint

tokenizer = AutoTokenizer.from_pretrained(base_model)
model = AutoModelForCausalLM.from_pretrained(base_model).to("cuda")
model = PeftModel.from_pretrained(model, adapter_path)
model = model.merge_and_unload().to("cuda")
model.eval()


2025-06-20 15:13:10.601155: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750432390.820483      67 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750432390.880402      67 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1e-05)
    (rotary_emb): 

# build_faiss_index

In [5]:
from datasets import load_dataset
import pandas as pd

# Replace this with your dataset if custom
hcm = load_dataset("lavita/ChatDoctor-HealthCareMagic-100k", split="train")

# Format for RAG
hcm_docs = [
    f"(HealthCareMagic)\nQ: {item['input']}\nA: {item['output']}"
    for item in hcm
]

In [6]:
pubmed = load_dataset("pubmed_qa", "pqa_labeled", split="train")
pubmed_docs = [
    f"(PubMedQA)\nQ: {row['question']} \nA: {row['long_answer']}"
    for row in pubmed
]

# Combine both
combined_docs = pubmed_docs + hcm_docs

README.md:   0%|          | 0.00/5.19k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [7]:
from sentence_transformers import SentenceTransformer
import faiss
import pickle

embedder = SentenceTransformer("all-MiniLM-L6-v2")  # or try all-mpnet-base-v2
doc_embeddings = embedder.encode(combined_docs, convert_to_numpy=True)

# Build index
index = faiss.IndexFlatL2(doc_embeddings.shape[1])
index.add(doc_embeddings)

# Save
faiss.write_index(index, "rag_index_combined.faiss")
with open("rag_corpus_combined.pkl", "wb") as f:
    pickle.dump(combined_docs, f)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/3537 [00:00<?, ?it/s]

# retriever

In [8]:
import faiss
import pickle
from sentence_transformers import SentenceTransformer

# Load retriever assets
embedder = SentenceTransformer("all-MiniLM-L6-v2")
index = faiss.read_index("rag_index_combined.faiss")
with open("rag_corpus_combined.pkl", "rb") as f:
    corpus = pickle.load(f)

def get_top_k_docs_combined(query, k=3):
    query_embedding = embedder.encode([query])
    D, I = index.search(query_embedding, k)
    return [combined_docs[i] for i in I[0]]


# RAG Prompt Construction

In [9]:
def build_rag_prompt(query, retrieved_docs):
    context = "\n".join(retrieved_docs)
    prompt = f"""<|user|>: Based on the following information from doctors and research:\n\n{context}\n\nAnswer this user query clearly and in simple terms:\n{query}\n\n<|assistant|>:"""
    return prompt

# inference

In [10]:
import torch

In [11]:
def rag_infer(query, model, tokenizer):
    docs = get_top_k_docs_combined(query)
    prompt = build_rag_prompt(query, docs)

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    with torch.no_grad():
        outputs = model.generate(
    **inputs,
    max_new_tokens=300,  # ⬅️ try 250–300
    temperature=0.7,
    top_p=0.9,
    pad_token_id=tokenizer.eos_token_id,
    eos_token_id=tokenizer.eos_token_id
)

    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return result.split("<|assistant|>:")[-1].strip()


In [12]:
query = "I have fever and sore throat for 2 days"
response = rag_infer(query, model, tokenizer)
print("🧠 RAG Response:", response)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


🧠 RAG Response: Based on the information provided, it seems that you have a fever and sore throat. The symptoms you have are:

- Fever: 103°F (40°C) or higher
- Sore throat
- Cough
- Headache
- Diarrhea
- Dizziness
- Stomach symptoms (e.g., abdominal pain, nausea, vomiting)

As per the information provided, the cause of your fever and sore throat is a viral infection. The most common viral infections that cause fever and sore throat are:

- Bacterial infections: If you have a bacterial infection, it can cause a sore throat, fever, and other symptoms.
- Viral infections: If you have a viral infection, it can cause a sore throat, fever, and other symptoms.

If you have a sore throat, it's best to see a doctor to rule out any underlying conditions. If you have a fever, it's best to see a doctor to rule out any underlying conditions.

If you have a sore throat and fever, it's best to take antibiotics to treat the bacterial infection. If you have a fever and


# Testing

In [13]:
test_queries = [
    "What are the symptoms of typhoid?",
    "I have body pain, headache and cold for 3 days.",
    "Explain this in simple words: acute bronchitis with respiratory distress",
    "How can I manage high blood pressure at home?",
    "I have chest pain when breathing deeply — what could be the reason?"
]

for i, query in enumerate(test_queries, 1):
    print(f"\n🔹 Test {i}")
    print(f"🧑‍💬 User Query: {query}")
    response = rag_infer(query, model, tokenizer)
    print(f"🧠 RAG Response:\n{response}")
    print("-" * 100)


🔹 Test 1
🧑‍💬 User Query: What are the symptoms of typhoid?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

🧠 RAG Response:
Sure, here are some common symptoms of typhoid:

1. Fever: The first and most obvious symptom of typhoid is a high fever. The temperature can range from 102°F to 104°F (39°C to 40°C).

2. Headache: Typhoid fever can cause headaches, which can be severe and last for several days.

3. Chills: Chills are a common symptom of typhoid, and they can last for several days.

4. Abdominal pain: Typhoid fever can cause abdominal pain, which can be severe and last for several days.

5. Nausea and vomiting: Typhoid fever can cause nausea and vomiting, which can be severe and last for several days.

6. Fatigue: Typhoid fever can cause fatigue, which can be severe and last for several days.

7. Jaundice: Typhoid fever can cause jaundice, which is a yellowing of the skin and eyes.

8. Diarrhea: Typhoid fever can cause diarrhea, which can be severe and last for several days.

9. Sore throat: Typhoid fever can cause a sore throat,
---------------------------------------------------------

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

🧠 RAG Response:
Based on the information provided, it seems that you have been experiencing symptoms of a viral illness, such as a sore throat, headaches, fever, chills, and weakness. These symptoms may be due to a viral infection, which can cause pain, headaches, fever, chills, and weakness. It is recommended to take anti-inflammatory medications such as ibuprofen to prevent fever and body-ache. It is also recommended to consult a physician for further diagnosis and treatment. It is essential to follow proper hygiene and self-care measures to prevent the spread of the virus. It is recommended to rest, drink plenty of fluids, and seek medical attention if the symptoms persist. It is also essential to follow the prescribed treatment plan to ensure a quick recovery.
----------------------------------------------------------------------------------------------------

🔹 Test 3
🧑‍💬 User Query: Explain this in simple words: acute bronchitis with respiratory distress


...

🧠 RAG Response:
Acute bronchitis is a respiratory infection that causes inflammation and narrowing of the bronchi, leading to shortness of breath and coughing. The symptoms of acute bronchitis include fever, cough, chest pain, and shortness of breath. The condition can be severe and lead to respiratory distress, which can cause shortness of breath and difficulty breathing. In severe cases, acute bronchitis can lead to pneumonia, which can be life-threatening. To diagnose acute bronchitis, a doctor will conduct a physical examination, including a chest x-ray and a pulmonary function test (PFT). If the doctor suspects that the patient has acute bronchitis, they will prescribe antibiotics, inhaled bronchodilators, and inhaled corticosteroids to help alleviate the symptoms and improve the patient's overall health. It is essential to seek medical attention if you experience severe shortness of breath, coughing, or chest pain, as these symptoms may indicate a more severe form of bronchitis.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

🧠 RAG Response:
Hi there! To manage high blood pressure at home, you can follow the following guidelines:

1. Eat a healthy diet: Avoid processed foods, high-sodium and high-sugar foods, and limit your intake of red meat, processed meats, and fried foods. Instead, opt for whole grains, fruits, vegetables, and lean proteins like chicken, fish, and beans.

2. Limit salt intake: Avoid adding salt to your food, as it can increase blood pressure. Instead, use herbs and spices to season your food, or consider using a salt substitute.

3. Exercise regularly: Regular exercise can help lower blood pressure by increasing blood flow to the heart and reducing stress. Try to get at least 30 minutes of moderate-intensity exercise most days of the week.

4. Manage stress: Stress can increase blood pressure, so try to manage stress through relaxation techniques like deep breathing, meditation, or yoga.

5. Limit alcohol intake: Alcohol can increase blood pressure, so try to limit your alcohol intake t

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

🧠 RAG Response:
Based on the information provided, it seems that the user has experienced chest pain when breathing deeply, which is a symptom of pleurisy. Pleurisy is an inflammation of the lining of the lungs, and it can cause chest pain on deep breathing. The pain is usually located on the left side (or behind the breastbone) and worsens while lying down and when lying down and deep breathing. The diagnosis is confirmed with EKG and Echocardiography. Pleurisy can also cause other symptoms, such as chest tightness or pain, which can be similar to chest pain on deep breathing. The user should consult a pulmonologist and get a clinical examination of the respiratory system, chest x-ray, and PFT (Pulmonary Function Test) to confirm the diagnosis. If pleurisy is the cause, the user should avoid heavyweight lifting and strenuous exercise, take simple painkillers like paracetamol or ibuprofen, and avoid movements causing pain. The user should also consult a pulmonologist and get further me

# rag_module.py

In [ ]:
import faiss
import pickle
from sentence_transformers import SentenceTransformer

# Load FAISS index + corpus
index = faiss.read_index("rag_index_combined.faiss")
with open("rag_corpus_combined.pkl", "rb") as f:
    corpus = pickle.load(f)

embedder = SentenceTransformer("all-MiniLM-L6-v2")

def get_top_k_docs(query, k=3):
    query_embedding = embedder.encode([query])
    D, I = index.search(query_embedding, k)
    return [corpus[i] for i in I[0]]

def build_rag_prompt(query, retrieved_docs):
    context = "\n".join(retrieved_docs)
    prompt = (
        f"<|user|>: Based on the following information from verified medical sources:\n\n"
        f"{context}\n\nAnswer this user query clearly and in simple terms:\n{query}\n\n<|assistant|>:"
    )
    return prompt

def rag_infer(query, model, tokenizer, max_new_tokens=300):
    docs = get_top_k_docs(query)
    prompt = build_rag_prompt(query, docs)
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.7,
            top_p=0.9,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id,
        )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)
